In [3]:
import numpy as np


In [8]:
def sigmoid(x):
    res = 1+np.exp(-x)
    return 1/res


In [9]:
def sigmoid_derivative(x):
    return np.exp(-x)/((1+np.exp(-x))**2)



In [104]:
class Cell(object):
    def __init__(self, d):
        self.w = np.random.rand(d)
        self.b = np.random.rand(1)
        self.input = 0
        self.output = 0
        self.E = 0
        self.old_w = self.w
        
    def front_pass(self, x):
        self.input = x.flatten()
        self.output = sigmoid(np.dot(self.w.T, x)+self.b)
        return self.output
    
    def update_W(self, e):
        self.old_w = self.w
        self.w = self.w - lr * e
    
    def bp_output(self, y):
        deta = -(y - self.output)*self.output*(1-self.output)*self.input
        self.E = -(y - self.output)*self.output*(1-self.output)
        self.update_W(deta)
        return self.E
        
    def bp_hidden(self, es, w):
        e_total = 0
        for i in w:
            for j in es:
                e_total = e_total + i*j
        self.E = e_total*self.output*(1-self.output)
        deta = np.multiply(self.E, self.input)
        self.update_W(deta)
        return deta
    
        
def L2_loss(y, yt):
    y.flatten()
    return np.sum(np.square(y-yt))    
    

In [ ]:
h1 = Cell(2)
h2 = Cell(2)

g1 = Cell(2)
g2 = Cell(2)
g3 = Cell(2)

o1 = Cell(3)

# layer1 = [h1, h2]
# layer2 = [o1, o2]
data = np.array([[0, 0],
                [0, 1],
                [1, 0],
                [1, 1]])

y = np.array([0,1,1,0])


In [60]:
lr = 0.5

In [ ]:
for j in range(100000):
    for i in range(4):
        oh1 = h1.front_pass(data[i])
        oh2 = h2.front_pass(data[i])
        
        x1 = np.array([oh1, oh2])
        # print("x1",x1)
        og1 = g1.front_pass(x1)
        og2 = g2.front_pass(x1)
        og3 = g3.front_pass(x1)
   
        x2 = np.array([og1, og2, og3])
        # print("x2",x2)
        oo1 = o1.front_pass(x2)
       
        if j % 5000 == 0:
            print("iter %d %f loss is %f"%(j,oo1,L2_loss(oo1, y[i])))

        oe1 = o1.bp_output(y[i])
        
        
        # Es = np.array(e1)
        
        ge1 = g1.bp_hidden(oe1,[o1.old_w[0]])
        ge2 = g2.bp_hidden(oe1,[o1.old_w[1]])
        ge3 = g3.bp_hidden(oe1,[o1.old_w[2]])
        
        ge = [ge1,ge2,ge3]
        w1 = [g1.old_w[0],g2.old_w[0],g3.old_w[0]]
        w2 = [g1.old_w[1],g2.old_w[1],g3.old_w[1]]
        he1 = h1.bp_hidden(ge, w1)
        he2 = h2.bp_hidden(ge, w2)



In [86]:
class Layer(object):
    # n 是神经元个数 d 是神经元维数
    def __init__(self, n, d):
        # 初始化n个神经元
        cells = []
        for i in range(n):
            cells.append(Cell(d))
        self.cells = cells
        self.n = n
        self.d = d
        
    def forward_propagation(self, x):
        res = []
        for i in range(self.n):
            res.append(self.cells[i].front_pass(x))
        return np.array(res)
    
    def back_propagation_output(self, y):
        # y的维数应该和n相同
        res = []
        for i in range(self.n):
            res.append(self.cells[i].bp_output(y[i]))
        return np.array(res)
    
    def back_propagation_hidden(self, e, w):
        # y的维数应该和n相同
        w = w.T
        # print("bph w ",w)
        res = []
        for i in range(self.n):
            res.append(self.cells[i].bp_hidden(e, w[i]))
        return np.array(res)
    
    def get_w(self):
        res = []
        for i in self.cells:
            res.append(i.old_w)
        return np.array(res)

In [87]:
layer1 = Layer(2, 2)
layer2 = Layer(3, 2)
layer3 = Layer(1, 3)

In [92]:
lr = 1

In [93]:

for j in range(100000):
    for i in range(4):
        fo1 = layer1.forward_propagation(data[i])
        fo2 = layer2.forward_propagation(fo1)
        fo3 = layer3.forward_propagation(fo2)
        
        if j % 5000 == 0:
            print("iter %d %f loss is %f"%(j,fo3,L2_loss(fo3, y[i])))
        
        be3 = layer3.back_propagation_output([y[i]])
        ow3 = layer3.get_w()
        
        be2 = layer2.back_propagation_hidden(be3, ow3)
        ow2 = layer2.get_w()
        
        be1 = layer1.back_propagation_hidden(be2, ow2)

iter 0 0.000000 loss is 0.000000
iter 0 0.612862 loss is 0.149876
iter 0 0.612638 loss is 0.150049
iter 0 0.022298 loss is 0.000497


iter 5000 0.000000 loss is 0.000000
iter 5000 0.612845 loss is 0.149889
iter 5000 0.612624 loss is 0.150061
iter 5000 0.018695 loss is 0.000350


iter 10000 0.000000 loss is 0.000000
iter 10000 0.612901 loss is 0.149846
iter 10000 0.612688 loss is 0.150011
iter 10000 0.018306 loss is 0.000335


iter 15000 0.000000 loss is 0.000000
iter 15000 0.612953 loss is 0.149805
iter 15000 0.612747 loss is 0.149965
iter 15000 0.017936 loss is 0.000322


iter 20000 0.000000 loss is 0.000000
iter 20000 0.613002 loss is 0.149768
iter 20000 0.612803 loss is 0.149922
iter 20000 0.017584 loss is 0.000309


...

iter 30000 0.000000 loss is 0.000000
iter 30000 0.613090 loss is 0.149700
iter 30000 0.612904 loss is 0.149844
iter 30000 0.016935 loss is 0.000287


iter 35000 0.000000 loss is 0.000000
iter 35000 0.613129 loss is 0.149669
iter 35000 0.612949 loss is 0.149808
iter 35000 0.016635 loss is 0.000277


iter 40000 0.000000 loss is 0.000000
iter 40000 0.613167 loss is 0.149640
iter 40000 0.612992 loss is 0.149775
iter 40000 0.016349 loss is 0.000267


iter 45000 0.000000 loss is 0.000000
iter 45000 0.613202 loss is 0.149613
iter 45000 0.613032 loss is 0.149744
iter 45000 0.016077 loss is 0.000258


iter 50000 0.000000 loss is 0.000000
iter 50000 0.613235 loss is 0.149587
iter 50000 0.613070 loss is 0.149714
iter 50000 0.015817 loss is 0.000250


In [137]:
def train(iter_num, layers, data_size=4, lr = 1, show_num = 5000):
    loss = 0
    for j in range(iter_num):
        # 现在没有对inputdata做封装
        for i in range(len(layers)):
            if i == 0:
                fp_res = layers[i].forward_propagation(data[j%data_size])
            else:
                fp_res = layers[i].forward_propagation(fp_res)
            if i == len(layers)-1:
                loss = loss+L2_loss(fp_res, y[j % data_size])
                if j % show_num == 0 :
                    print("iter %d loss is %f" % (j, loss/show_num))
                    loss = 0
        for i in range(len(layers)):
            if i == 0:
                bp_res = layers[len(layers)-i-1].back_propagation_output([y[j % data_size]])
            else:
                bp_res = layers[len(layers)-i-1].back_propagation_hidden(bp_res, old_w)
            old_w = layers[len(layers)-i-1].get_w()
            

In [140]:
def test(layers, test_data):
    fp_res = 0
    for i in range(len(layers)):
            if i == 0:
                fp_res = layers[i].forward_propagation(test_data)
            else:
                fp_res = layers[i].forward_propagation(fp_res)
    print("test result is",fp_res)

In [149]:
layer1 = Layer(2, 2)
layer2 = Layer(3, 2)
layer3 = Layer(1, 3)
net = [layer1, layer2, layer3]


In [153]:
train(100000, net)

iter 0 loss is 0.000000


iter 5000 loss is 0.073447


iter 10000 loss is 0.072862


iter 15000 loss is 0.072357


iter 20000 loss is 0.071916


...

iter 75000 loss is 0.069262


iter 80000 loss is 0.069127


iter 85000 loss is 0.069001


iter 90000 loss is 0.068884


iter 95000 loss is 0.068775


In [152]:
test(net, np.array([1,0]))
test(net, np.array([0,0]))

test result is [[0.618406]]
test result is [[2.70503012e-07]]
